In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
791,This is a great British film. A cleverly obser...,positive
358,Where oh where to begin in describing the comp...,negative
53,Perhaps once in a generation a film comes alon...,positive
638,"I would just like it to be known, that I do no...",negative
655,This is only the second time I stopped a video...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
791,great british film cleverly observed script ma...,positive
358,oh begin describing comprehensive wretchedness...,negative
53,perhaps generation film come along perfection ...,positive
638,would like known often rate movie originally e...,negative
655,second time stopped video dvd part way through...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
791,great british film cleverly observed script ma...,1
358,oh begin describing comprehensive wretchedness...,0
53,perhaps generation film come along perfection ...,1
638,would like known often rate movie originally e...,0
655,second time stopped video dvd part way through...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/LuckyChauhan18/Capstone-project-Sentiment-Analysis-.mlflow')
dagshub.init(repo_owner='LuckyChauhan18', repo_name='Capstone-project-Sentiment-Analysis-', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as LuckyChauhan18

Initialized MLflow to track repo "LuckyChauhan18/Capstone-project-Sentiment-Analysis-"

Repository LuckyChauhan18/Capstone-project-Sentiment-Analysis- initialized!

<Experiment: artifact_location='mlflow-artifacts:/93e79a1a3b7c47e7884907814d83a361', creation_time=1763412258980, experiment_id='0', last_update_time=1763412258980, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-18 02:53:34,200 - INFO - Starting MLflow run...
2025-11-18 02:53:34,832 - INFO - Logging preprocessing parameters...
2025-11-18 02:53:36,356 - INFO - Initializing Logistic Regression model...
2025-11-18 02:53:36,357 - INFO - Fitting the model...
2025-11-18 02:53:36,400 - INFO - Model training complete.
2025-11-18 02:53:36,401 - INFO - Logging model parameters...
2025-11-18 02:53:36,866 - INFO - Making predictions...
2025-11-18 02:53:36,867 - INFO - Calculating evaluation metrics...
2025-11-18 02:53:36,879 - INFO - Logging evaluation metrics...
2025-11-18 02:53:38,407 - INFO - Saving and logging the model...
2025/11/18 02:53:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-11-18 02:53:54,073 - INFO - Model training and logging completed in 19.24 seconds.
2025-11-18 02:53:54,074 - INFO - Accuracy: 0.664
2025-11-18 02:53:54,076 - INFO - Precision: 

🏃 View run hilarious-panda-629 at: https://dagshub.com/LuckyChauhan18/Capstone-project-Sentiment-Analysis-.mlflow/#/experiments/0/runs/0292aec1f0134285a140ee0b2696f694
🧪 View experiment at: https://dagshub.com/LuckyChauhan18/Capstone-project-Sentiment-Analysis-.mlflow/#/experiments/0
